# Análisis y Limpieza de Dataset Fintech

En este notebook se realiza un proceso de análisis exploratorio y limpieza de datos sobre un dataset del sector financiero, incluyendo tratamiento de valores nulos, detección de outliers y análisis de correlaciones.

In [1]:
import pandas as pd
import numpy as np

## 1. Carga del Dataset

Se carga el dataset generado previamente con valores nulos y posibles valores atípicos para su posterior análisis.

In [2]:
df = pd.read_csv("dataset_fintech_sucio.csv")
df.head()

,cliente_id,edad,ingreso_mensual,monto_credito,score_crediticio,tipo_cliente,tiene_mora
0,1,56,1.330539e+06,1.828952e+06,NaN,Recurrente,0
1,2,69,1.058116e+06,4.264370e+06,357.0,Premium,0
2,3,46,4.201575e+05,2.000000e+07,558.0,Premium,0
3,4,32,6.789415e+05,6.158858e+06,641.0,Nuevo,0
4,5,60,1.116728e+06,3.591981e+06,429.0,Premium,1


## 2. Exploración Inicial del Dataset

Se analiza la estructura del dataset, tipos de datos y presencia de valores nulos.

In [3]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cliente_id        500 non-null    int64  
 1   edad              500 non-null    int64  
 2   ingreso_mensual   480 non-null    float64
 3   monto_credito     500 non-null    float64
 4   score_crediticio  485 non-null    float64
 5   tipo_cliente      500 non-null    str    
 6   tiene_mora        500 non-null    int64  
dtypes: float64(3), int64(3), str(1)
memory usage: 27.5 KB


## 3. Tratamiento de Valores Nulos

Se identifican valores faltantes en las variables `ingreso_mensual` y `score_crediticio`.  
Dado que el porcentaje es bajo, se decide imputar utilizando la mediana para evitar distorsión por valores extremos.

In [4]:
df.isnull().sum()

cliente_id           0
edad                 0
ingreso_mensual     20
monto_credito        0
score_crediticio    15
tipo_cliente         0
tiene_mora           0
dtype: int64

In [6]:
# Imputar ingreso_mensual con mediana
df["ingreso_mensual"] = df["ingreso_mensual"].fillna(df["ingreso_mensual"].median())

# Imputar score_crediticio con mediana
df["score_crediticio"] = df["score_crediticio"].fillna(df["score_crediticio"].median())

In [7]:
df.isnull().sum()

cliente_id          0
edad                0
ingreso_mensual     0
monto_credito       0
score_crediticio    0
tipo_cliente        0
tiene_mora          0
dtype: int64

In [8]:
df.describe()

,cliente_id,edad,ingreso_mensual,monto_credito,score_crediticio,tiene_mora
count,500.000000,500.000000,5.000000e+02,5.000000e+02,500.000000,500.000000
mean,250.500000,44.220000,1.279159e+06,3.321858e+06,579.410000,0.228000
std,144.481833,15.036082,4.907531e+06,2.053354e+06,156.802065,0.419963
min,1.000000,18.000000,-1.031684e+04,-4.755065e+05,306.000000,0.000000
25%,125.750000,32.000000,6.247800e+05,2.333677e+06,446.500000,0.000000
50%,250.500000,45.000000,7.928975e+05,3.156644e+06,569.000000,0.000000
75%,375.250000,57.000000,9.528298e+05,3.953365e+06,713.500000,0.000000
max,500.000000,69.000000,5.000000e+07,2.000000e+07,849.000000,1.000000


In [9]:
Q1 = df["monto_credito"].quantile(0.25)
Q3 = df["monto_credito"].quantile(0.75)
IQR = Q3 - Q1

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

limite_superior

np.float64(6382896.057520328)

## 4. Detección y Tratamiento de Valores Atípicos

Se utiliza el método del rango intercuartílico (IQR) para identificar valores extremos en la variable `monto_credito`.  
Los valores que superan el límite superior estadístico son tratados mediante winsorización.

In [10]:
outliers = df[df["monto_credito"] > limite_superior]
outliers

,cliente_id,edad,ingreso_mensual,monto_credito,score_crediticio,tipo_cliente,tiene_mora
2,3,46,420157.508511,20000000.0,558.0,Premium,0
39,40,69,676749.766335,20000000.0,759.0,Premium,0
58,59,24,745579.699193,20000000.0,615.0,Nuevo,0
281,282,42,497245.950059,20000000.0,762.0,Premium,0
348,349,68,989247.154161,20000000.0,356.0,Recurrente,0


Se detectaron 5 valores atípicos en la variable "monto_credito" utilizando el método del rango intercuartílico (IQR).
Estos valores (20.000.000) superan el límite superior estadístico (~6.38 millones), por lo que se decide aplicar winsorización, reemplazándolos por el límite superior para evitar distorsión en los análisis posteriores sin eliminar registros.

In [ ]:
df.loc[df["monto_credito"] > limite_superior, "monto_credito"] = limite_superior

In [12]:
df["monto_credito"].max()

np.float64(6382896.057520328)

## 5. Análisis de Relaciones entre Variables

Se analiza la correlación entre variables numéricas para identificar posibles relaciones relevantes para el comportamiento crediticio.

In [13]:
df.corr(numeric_only=True)

,cliente_id,edad,ingreso_mensual,monto_credito,score_crediticio,tiene_mora
cliente_id,1.000000,0.061964,-0.003924,-0.074349,-0.081487,0.042308
edad,0.061964,1.000000,0.092959,-0.031432,-0.123395,-0.002882
ingreso_mensual,-0.003924,0.092959,1.000000,0.013678,-0.071131,0.040627
monto_credito,-0.074349,-0.031432,0.013678,1.000000,-0.048057,-0.038742
score_crediticio,-0.081487,-0.123395,-0.071131,-0.048057,1.000000,-0.026681
tiene_mora,0.042308,-0.002882,0.040627,-0.038742,-0.026681,1.000000


### Interpretación de Correlaciones

Las correlaciones entre variables numéricas son bajas (cercanas a 0), lo que indica ausencia de relaciones lineales fuertes.

La relación más destacable es entre `edad` y `score_crediticio` (-0.12), aunque su magnitud sigue siendo débil.

En general, el comportamiento de mora (`tiene_mora`) no muestra una asociación lineal significativa con ingreso, monto de crédito o score crediticio.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'seaborn'